In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from rl_rnn import RLRNN
from rl_rnn_mem import RLRNNMEM
from ops import Operations
from session import *
from data_gen import *
import pickle
from functools import reduce


path = "/home/user/projects/neural_program_synthesis/models/summaries/1sample/RLRNN/np_center-5ops/total_num_epochs#80000~state_size#200~test_ratio#0.5~num_samples#2~batch_size#1~learning_rate#0.005~epsilon#0.001~num_features#4~state_fn#relu~pen_sofmax#False~augument_grad#False~max_reward#1000~relaunch#True~seed#53375"
test_1000 = False
old = False

model_path = path+'/model'
cfg_path = path+'/cfg.p'

#get the global configuration
cfg = pickle.load(open(cfg_path, 'rb'))


#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if cfg['model'] == "RNN":
        ops = Operations(cfg)
        if 'used_ops_obj' in cfg:
                ops.ops = cfg["used_ops_obj"]
                ops.num_of_ops = len(ops.ops)
        if 'used_ops_obj_mem' in cfg:
                ops.ops_mem = cfg["used_ops_obj_mem"]
                ops.num_of_ops_mem = len(ops.ops_mem)
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path, test_1000)
elif cfg['model'] == "HistoryRNN":
        ops = Operations(cfg)
        ops.ops = cfg["used_ops_obj"]
        ops.num_of_ops = len(ops.ops)
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path, test_1000)
elif cfg['model'] == "RLRNN":
        ops_env = OpsEnv(cfg)
        if 'used_ops_env' in cfg:
                ops_env = cfg["used_ops_env"]
        mem = RLRNNMEM(cfg, ops_env) 
        model = RLRNN(cfg, ops_env, mem) 
        res = restore_selection_RL_RNN(model, cfg, x_train, x_test, y_train, y_test, model_path, test_1000)
else:
        raise Exception('did not find the model')

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


norming the grads
grads are
[(<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_0:0' shape=(204, 200) dtype=float64>, <tf.Variable 'W_mem:0' shape=(204, 200) dtype=float64_ref>), (<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_1:0' shape=(200,) dtype=float64>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(200,) dtype=float64_ref>), (<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_2:0' shape=(200, 4) dtype=float64>, <tf.Variable 'W2_mem:0' shape=(200, 4) dtype=float64_ref>), (<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_3:0' shape=(4,) dtype=float64>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(4,) dtype=float64_ref>), (<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_4:0' shape=(4, 4) dtype=float64>, <tf.Variable 'W3_mem:0' shape=(4, 4) dtype=float64_ref>), (<tf.Tensor 'RNN_mem/clip_by_global_norm/RNN_mem/clip_by_global_norm/_5:0' shape=(4,) dtype=float64>, <tf.Variable 'RNN_mem/P

In [2]:
def print_errors(error_lst, name):
    print("")
    print("Total", reduce((lambda x, y: x + y), error_lst))
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            if len(matrix[elem]) < 2:
                index = matrix[elem][0]
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [6]:
#print_errors(res["total_loss_traind_train"], "sofmax_train_error")
#print_errors(res["total_loss_traind_test"], "hardmax_train_error")
#print_errors(res["total_loss_testd_train"], "softmax_test_error")
#print_errors(res["total_loss_testd_test"], "hardmax_test_error")

print_errors(res["train_math_error"], "train_math_error")
print_errors(res["test_math_error"], "test_math_error")


Total 220.939952266
train_math_error[0] err is 220.939952266

Total 0.0
test_math_error[0] err is 0.0


In [9]:
#Softmax Hardmax Print
#print_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0])
#print_ops_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0], cfg["used_ops_obj"])

##RL print - train OP selections
print_matrix(res["test_math_error"][0], res["test_selections"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections"][0], cfg["used_ops_env"].ops)

Error for this matrix is 0.0

##Elem-0--#############################################################
[4] [4] [3] [4] 
Error for this matrix is 0.0

##Elem-0--#############################################################
[ np_sub_env ] [ np_sub_env ] [ np_div_env ] [ np_sub_env ] 


In [10]:
#Softmax Hardmax Print

#print_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0])
#print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"])

##RL print - train MEM selections
print_matrix(res["test_math_error"][0], res["test_selections_mem"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections_mem"][0], cfg["used_ops_env"].ops_mem)

Error for this matrix is 0.0

##Elem-0--#############################################################
[3] [0] [2] [3] 
Error for this matrix is 0.0

##Elem-0--#############################################################
[ np_stall_env ] [ np_add_env ] [ np_get_size_env ] [ np_stall_env ] 


In [14]:
res['test_current_exes']

[[array([[ 29.089, -88.536,  20.493, -94.298]]),
  array([[ 447.356,  -59.463,  -49.153,  103.597]]),
  array([[ 421.424,  469.706, -568.444,  621.959]]),
  array([[-1739.944,  1072.938,  -587.89 ,    41.647]])]]

In [17]:
res['test_outputs']

[[array([[   0.   , -117.625,   -8.596, -123.387]]),
  array([[-133.251,    0.   ,    0.   ,    0.   ]]),
  array([[-33.313,   0.   ,   0.   ,   0.   ]]),
  array([[ 62.402, -55.223,  53.806, -60.985]])]]

In [18]:
res['test_outputs_mem']

[[array([[ 29.089, -88.536,  20.493, -94.298]]),
  array([[-133.251,    0.   ,    0.   ,    0.   ]]),
  array([[4, 0, 0, 0]]),
  array([[ 29.089, -88.536,  20.493, -94.298]])]]

In [15]:
res['batchesY_test']

[array([[ 62.402, -55.223,  53.806, -60.985]])]

In [16]:
res['batchesX_test']

[array([[ 29.089, -88.536,  20.493, -94.298]])]

In [13]:
res[]

dict_keys(['batchesY_train', 'test_selections_mem', 'train_states', 'train_current_exes', 'test_outputs_mem', 'test_selections', 'test_outputs', 'train_states_mem', 'batchesX_train', 'batchesY_test', 'train_selections', 'test_rewards', 'batchesX_test', 'test_current_exes_mem', 'train_selections_mem', 'test_states', 'train_rewards', 'test_discount_rewards', 'test_math_error', 'test_states_mem', 'train_math_error', 'test_current_exes', 'train_outputs', 'train_current_exes_mem', 'train_discount_rewards', 'train_outputs_mem'])

In [9]:
#Softmax Hardmax Print

print_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], ops.ops_mem )

##RL print - train MEM selections
#print_matrix(res["train_math_error"][0], res["train_selections_mem"][0])
#print_ops_matrix(res["train_math_error"][0], res["train_selections_mem"][0], cfg["used_ops_env"].ops_mem)

Error for this matrix is 2.52435489671e-29

##Elem-0--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 0.  0.  1.  0.] [ 0.  0.  1.  0.] [ 0.  0.  1.  0.] 
Error for this matrix is 2.52435489671e-29

##Elem-0--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_stall ] [ tf_stall ] [ tf_stall ] 


In [ ]:
##RL print - test OP selections
print_matrix(res["test_math_error"][0], res["test_selections"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections"][0], cfg["used_ops_env"].ops)

In [ ]:
##RL print - test MEM selections
print_matrix(res["test_math_error"][0], res["test_selections_mem"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections_mem"][0], cfg["used_ops_env"].ops_mem)

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

In [ ]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res['last_hardmax_state_train_mem']

In [ ]:
cfg['num_features']

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
#res_pred = predict_form_sess(model, cfg, x_test[0:100,], res['last_hardmax_state_train'],res['last_hardmax_state_train_mem'], path = model_path, mode="hard")

In [ ]:
x[0:1]

In [ ]:
x_test[0:100,]

In [ ]:
res_pred

In [ ]:
len(res["softmaxes_traind_train"])

In [ ]:
test_1000_samples_RL(model, res['sess'],92964)

In [ ]:
res.keys()

In [ ]:
res['test_discount_rewards']